In [1]:
import igl # Library to load meshes and perform operations on them
import meshplot as mp # Library to visualize meshes and point clouds
import vedo as vd # Library to visualize meshes and point clouds
import polyscope as ps # Library to visualize meshes
import numpy as np # Library to perform operations on matrices
import os # Library to perform operations on files and directories
import matplotlib.pyplot as plt
from math import pi
import sys

# Importing the classes and functions from the visualization folder
vd.settings.default_backend = 'k3d'

# Directory path
dir_path = os.getcwd()


# Add hananLab to path

from pathlib import Path


currect_dir = Path.cwd()
# Obtain the path HananLab
path = os.path.dirname(currect_dir.parent)
sys.path.append(path)
print(path)


from hanan.geometry.utils import *
from hanan.optimization.Center_absolute import Center_abs
from hanan.optimization.Optimizer import Optimizer




c:\Users\aikyna\Desktop\hananLab


In [2]:
v, f = igl.read_triangle_mesh("C:/Users/aikyna/Desktop/hananLab/hJupyter/models/Objs/obj_tri/Hall.obj")
V = len(v) # Number of vertices
F = len(f) # Number of faces

v1, v2, k1, k2 = igl.principal_curvature(v, f)
normals = igl.per_vertex_normals(v, f)
h2 = 0.5 * (k1 + k2)

rads_min, rads_max = np.zeros(V), np.zeros(V)

for v_ind in range(V):
    rads_min[v_ind] = 1 / abs(k1[v_ind])
    rads_max[v_ind] = 1 / abs(k2[v_ind])
    if (rads_min[v_ind] > rads_max[v_ind]):
        rads_min[v_ind], rads_max[v_ind] = rads_max[v_ind], rads_min[v_ind]
        
rads = np.zeros(V)
for v_ind in range(V):
    rads[v_ind] = 1.0 / abs(h2[v_ind])

centers = np.zeros((V, 3))
for v_ind in range(V):
    centers[v_ind] = v[v_ind] + normals[v_ind] / h2[v_ind]

In [3]:
center_abs_init = np.zeros(3)

matrix_s = np.zeros((3 * V, 3))
vector_b = np.zeros(3*V)
for v_ind in range(V):
    normal = normals[v_ind]
    matrix_s[3 * v_ind, :] = np.array([0, -normal[2], normal[1]])
    matrix_s[3 * v_ind + 1, :] = np.array([normal[2], 0, -normal[0]])
    matrix_s[3 * v_ind + 2, :] = np.array([-normal[1], normal[0], 0])
    vector_b[3 * v_ind:3 * v_ind + 3] = matrix_s[3 * v_ind:3 * v_ind + 3, :] @ v[v_ind]
#print(matrix_s.T@matrix_s)
#print(matrix_s.T@vector_b)

center_abs_init = np.linalg.lstsq(matrix_s.T@matrix_s, matrix_s.T@vector_b, rcond=None)[0]
print(center_abs_init)

[18.97791031 26.11760639 -4.87671118]


In [4]:
vector = np.linalg.lstsq(normals, np.ones(V), rcond=None)[0]
vector = vector/np.linalg.norm(vector)
print(vector)

[-0.0101157  -0.56749374  0.82331557]


In [5]:
p = mp.plot(v, f, return_plot=True);
p.add_lines(v, v + normals, shading={"line_color": "red"});
p.add_points(np.array([center_abs_init]), shading={"point_color": "green", "point_size": 10.0});


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(19.867670…

In [6]:
from hanan.optimization import Center_absolute


J = np.zeros((V, 3))
r = np.zeros(V)
X_0 = center_abs_init.copy()

p_length = np.linalg.norm(center_abs_init - v, axis= 1)

w = np.ones(V)

# Optimizer
opt = Optimizer()

opt.add_variable("m", 3)
opt.initialize_optimizer(X_0, "LM", 0.7, 1)

orthog = Center_abs()


opt.add_constraint(orthog, args=(v, normals, p_length, w))

print(opt.constraints[0].var)
print(opt.constraints[0].const)



for _ in range(100):

    opt.get_gradients()
    opt.optimize()
    
X = opt.uncurry_X("m")
X0 = opt.X0 

print(X0, X)
# p_length = np.zeros(V)
# for v_ind in range(V):
#     p_length[v_ind] = np.linalg.norm(center_abs_init - v[v_ind])

# w = np.zeros(V)
# for v_ind in range(V):
#     angle = np.arccos(np.dot(v[v_ind] - center_abs_init, normals[v_ind]) / np.linalg.norm(v[v_ind] - center_abs_init))
#     w[v_ind] = np.tan(angle / 2)


3
3330
 E 1: 271.2100834147763	 dx: 7.946725614074065
[ 19.91892801  28.18590862 -12.49163466]
 E 2: 176.2179806767254	 dx: 2.444523453104904
[ 19.23714648  30.34182007 -13.42056666]
 E 3: 170.97769357780183	 dx: 0.631115611120207
[ 19.24952703  29.74870974 -13.20522095]
 E 4: 170.75876543688256	 dx: 0.21863265092558454
[ 19.28374896  29.95704657 -13.26200879]
 E 5: 170.7331923192258	 dx: 0.0775838859909649
[ 19.26460886  29.88266612 -13.2510332 ]
 E 6: 170.72991167743032	 dx: 0.028775436457884516
[ 19.27299243  29.91019207 -13.25078139]
 E 7: 170.72943949938787	 dx: 0.011122875623042277
[ 19.26952998  29.89984291 -13.25293191]
 E 8: 170.7293673815283	 dx: 0.004559438623717469
[ 19.27094608  29.90378989 -13.25114179]
 E 9: 170.72935552728677	 dx: 0.0020000218951490762
[ 19.27036054  29.90226054 -13.25228998]
 E 10: 170.72935338969762	 dx: 0.0009292278339335415
[ 19.27060706  29.90286299 -13.25162684]
 E 11: 170.7293529691147	 dx: 0.00044857293131719795
[ 19.270501    29.90262099 -13.25

In [7]:
print(center_abs_init, X)
p = mp.plot(v, f, return_plot=True);
p.add_lines(v, v + normals, shading={"line_color": "red"});
p.add_points(np.array([center_abs_init, X]), shading={"point_color": "green", "point_size": 10.0});

[18.97791031 26.11760639 -4.87671118] [ 19.27053325  29.90269106 -13.25186324]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(19.867670…

In [8]:
sum1 = 0
sum2 = 0
for v_ind in range(V):
    sum1 += (np.dot(normals[v_ind], v[v_ind] - center_abs_init) / np.linalg.norm(v[v_ind] - center_abs_init) - 1) ** 2
    sum2 += (np.dot(normals[v_ind], v[v_ind] - opt_point) / np.linalg.norm(v[v_ind] - opt_point) - 1) ** 2
print(sum1, sum2)


NameError: name 'opt_point' is not defined

In [ ]:
center_abs = X

In [ ]:
rad_init_min = abs(np.linalg.norm(center_abs - centers[0]) - rads[0])
rad_init_max = np.linalg.norm(center_abs - centers[0]) + rads[0]
for v_ind in range(V):
    rad_init_min = max(rad_init_min, abs(np.linalg.norm(center_abs - centers[v_ind]) - rads[v_ind]))
    rad_init_max = min(rad_init_max, np.linalg.norm(center_abs - centers[v_ind]) + rads[v_ind])

print("Radius minimum = ", rad_init_min)
print("Radius maximum = ", rad_init_max)

rad_abs = (rad_init_max + rad_init_min) * 0.5

Radius minimum =  nan
Radius maximum =  nan


In [ ]:
J = np.zeros((2 * V, 1 + 2 * V))
r = np.zeros(2 * V)
X_0 = np.concatenate(([rad_abs * rad_abs], np.random.rand(2*V)))

In [ ]:
def compute_J(v, normals, center_abs, rads_min, rads_max, X):
    rad_var = X[0]
        
    for v_ind in range(V):
        mu_1 = X[2 * v_ind + 1]
        mu_2 = X[2 * v_ind + 2]

        J[v_ind, 0] = 1 / np.dot(center_abs - v[v_ind], normals[v_ind]) / 2
        J[v_ind, 2 * v_ind + 1] = 2 * mu_1
        
        J[V + v_ind, 0] = 1 / np.dot(center_abs - v[v_ind], normals[v_ind]) / 2
        J[V + v_ind, 2 * v_ind + 2] = -2 * mu_2
        
        r[v_ind] = (rad_var - np.dot(center_abs - v[v_ind], center_abs - v[v_ind])) / (2 * np.dot(center_abs - v[v_ind], normals[v_ind])) - rads_max[v_ind] + mu_1 ** 2
        r[V + v_ind] = (rad_var - np.dot(center_abs - v[v_ind], center_abs - v[v_ind])) / (2 * np.dot(center_abs - v[v_ind], normals[v_ind])) - rads_min[v_ind] - mu_2 ** 2
        

In [ ]:
# from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, linalg

# # Compute pseudo Hessian
# X = X_0

# for i in range(15):
#     compute_J(v, normals, center_abs, rads_min, rads_max, X)    

#     H = J.T@J

#     H[np.diag_indices_from(H)] += np.diag(H).max()*1e-6

#     # Sparse matrix H
#     H = csc_matrix(H)

#     # Solve for dx
#     dx = linalg.spsolve(H, -J.T@r)
#     # Update vertices
#     X = X + 0.9*dx
    
#     # energy
#     energy = r.T@r
#     print(f"energy: {energy}\t dx: {np.linalg.norm(dx)}")
# print(X[0])


In [ ]:
# rad_abs = np.sqrt(X[0])

# rads = np.zeros(V)
# for v_ind in range(V):
#     rads[v_ind] = (rad_abs ** 2 - np.dot(center_abs - v[v_ind], center_abs - v[v_ind])) / np.dot(center_abs - v[v_ind], normals[v_ind]) / 2
#     if (rads[v_ind] < 0):
#         print(v_ind)
# print(rads)

# centers = np.zeros((V, 3))
# for v_ind in range(V):
#     centers[v_ind] = v[v_ind] - normals[v_ind] * rads[v_ind]

# max_dif = abs(rads[0] - 1 / abs(h2[0]))
# min_dif = abs(rads[0] - 1 / abs(h2[0]))
# for v_ind in range(V):
#     max_dif = max(max_dif, abs(rads[v_ind] - 1 / abs(h2[v_ind])))
#     min_dif = max(min_dif, abs(rads[v_ind] - 1 / abs(h2[v_ind])))
# print(min_dif, max_dif)

In [ ]:
# ps.init()
# ps.remove_all_structures()
 
# mesh = ps.register_surface_mesh("Mesh", v, f, smooth_shade=True)

# for _ in range(10):
#     i = np.random.randint(0, V-1)
#     c = centers[i] 
#     sphere = ps.register_point_cloud(f"sphere_c{i}", np.array([c]), enabled=True, color=(0,0,0), transparency=0.3)
#     sphere.set_radius(rads[i], relative=False)

# abs_sphere = ps.register_point_cloud(f"abs_sphere", np.array([center_abs]), enabled=True, color=(0,0,256), transparency=0.9)
# abs_sphere.set_radius(rad_abs, relative=False)

# ps.show()